In [45]:
import sys  
sys.path.insert(0, '../../pycurvis')
from data.vec_io import *
from data.vis_io import *
import vtk
from vtkmodules.util import numpy_support

def getijk(n, strides):
  k = n // strides[2]
  n -= k*strides[2]
  j = n // strides[1]
  n -= j*strides[1]
  i = n // strides[0]
  n -= i*strides[0]
  return [i, j, k]

In [2]:
sgr = vtk.vtkXMLStructuredGridReader()
sgr.SetFileName("data/bc80-45000-down.vts")
sgr.Update()
sg = sgr.GetOutput()

In [6]:
pd = sg.GetPointData()
theta = pd.GetArray(0)
print(theta.GetTuple(21241))
print(sg.GetBounds())

(300.21173095703125,)
(0.0, 696.0, -250.0, 150.0, 0.7504535071217414, 429.4370158281313)


In [118]:
# query = [0, 0, 0]
query = [0+200, -250.0+5, 0.7505+5]
pcoord = [0,0,0]
weights = [0,0,0,0,0,0,0,0]
subid = vtk.reference(0)
spid = sg.FindCell(query, None, 0, 1e-4, subid, pcoord, weights)
strides = [1, 88, 88*51]

i = [0, 1, 1, 0, 0, 1, 1, 0]
j = [0, 0, 1, 1, 0, 0, 1, 1]
k = [0, 0, 0, 0, 1, 1, 1, 1]
if spid > -1:
  for pidx in range(8):
    tmpspid = spid + i[pidx]*strides[0] + j[pidx]*strides[1] + k[pidx]*strides[2]
    print(tmpspid, getijk(tmpspid, strides), sg.GetPoint(tmpspid))

63325 [53, 5, 14] (424.0, -210.0, 126.73634439666756)


In [117]:
pcoord

[-9.375, 0.625, -79.46451548542096]

In [115]:
subid.get()

0

In [94]:
pcoord

[-5.0, 1.0, -41.403590523999455]

In [100]:
tmpGCell = vtk.vtkGenericCell()

# query = [0, 0, 0]
query = [10, -50.0, 100]
pcoord = [0,0,0]
weights = [0,0,0,0,0,0,0,0]
subid = vtk.reference(0)
spid = sg.FindCell(query, None, tmpGCell, 0, 0, subid, pcoord, weights)
print(tmpGCell.GetNumberOfPoints())
pids = tmpGCell.GetPointIds()
strides = [1, 88, 88*51]
for i in range(8):
  print(pids.GetId(i), getijk(pids.GetId(i), strides), sg.GetPoint(pids.GetId(i)))

8
2118 [6, 24, 0] (48.0, -58.0, 175.10739521681703)
2119 [7, 24, 0] (56.0, -58.0, 171.1106758878864)
2207 [7, 25, 0] (56.0, -50.0, 171.1106758878864)
2206 [6, 25, 0] (48.0, -50.0, 175.10739521681703)
6606 [6, 24, 1] (48.0, -58.0, 177.58077417243973)
6607 [7, 24, 1] (56.0, -58.0, 173.59741543316198)
6695 [7, 25, 1] (56.0, -50.0, 173.59741543316198)
6694 [6, 25, 1] (48.0, -50.0, 177.58077417243973)


In [87]:
pcoord

[0.0, 0.0, 0.373899810228308]

In [97]:
vtkInterp = vtk.vtkProbeFilter()
vtkInterp.SetSourceData(sg)
seeds_dataset = vtk.vtkPointSet()
seed = vtk.vtkPoints()
seed.InsertNextPoint([10, -50.0, 100])
seeds_dataset.SetPoints(seed)
vtkInterp.SetInputData(seeds_dataset)
vtkInterp.Update()
pc = vtkInterp.GetOutput()

In [99]:
values = numpy_support.vtk_to_numpy(pc.GetPointData().GetArray(0))
values

array([0.], dtype=float32)

In [54]:
sg.GetPoint(0)

(0.0, -250.0, 191.09518730899723)

In [34]:
print(sg.GetDimensions())
print(sg.GetExtent())
print(sg.GetBounds())
print(88*51)

(88, 51, 21)
(0, 87, 0, 50, 0, 20)
(0.0, 696.0, -250.0, 150.0, 0.7504535071217414, 429.4370158281313)
4488


In [ ]:
sg.GetDimensions

In [24]:
strides = [1, 88, 88*51]
pids = tmpGCell.GetPointIds()
for i in range(8):
  print(pids.GetId(i), sg.GetPoint(pids.GetId(i)))
  print(getijk(pids.GetId(i), strides))

1598 (112.0, -106.0, 143.1336405853721)
0
[14, 18, 0]
1599 (120.0, -106.0, 139.13692125644147)
0
[15, 18, 0]
1687 (120.0, -98.0, 139.13692125644147)
0
[15, 19, 0]
1686 (112.0, -98.0, 143.1336405853721)
0
[14, 19, 0]
6086 (112.0, -106.0, 145.71390425821784)
0
[14, 18, 1]
6087 (120.0, -106.0, 141.7305455189401)
0
[15, 18, 1]
6175 (120.0, -98.0, 141.7305455189401)
0
[15, 19, 1]
6174 (112.0, -98.0, 145.71390425821784)
0
[14, 19, 1]


In [27]:
vtk.reference([0,0,0])

vtkmodules.vtkCommonCore.tuple_reference([0, 0, 0])

In [26]:
phys = read_vec("data/mantle_phys.vec")

In [28]:
phys.shape

(201, 360, 140, 3)